In [ ]:
# %pip install sdv

### TPS과 Latency에 대해서 각각 다른 scaler 사용하기

### Data Prepare (Metric)

In [1]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

A_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")

In [2]:
A_metrics = pd.read_csv("/home/sein/mk_config/ycsb_AA/results/external_metrics.csv")

metrics = A_metrics.drop(['Unnamed: 0'], axis = 1)


In [3]:
metrics = metrics.replace([np.inf],9999999)

metrics


,tps,latency
0,0.216666,20847397
1,0.322221,12744942
2,3789.313102,1076
3,0.166667,12510551
4,0.266666,10686198
...,...,...
995,0.138889,11836756
996,0.149999,16386967
997,0.227778,19293708
998,0.188888,10529195


In [4]:
print(metrics['tps'].min() , metrics['tps'].max())
print(metrics['latency'].min() , metrics['latency'].max())


0.1055552118015207 4163.050339357126
898 22452142


### Data Prepare (Config)

In [5]:
import random
import glob
import os
import sys
import pandas as pd

knob_list = glob.glob("/home/sein/mk_config/ycsb_AA/configs/my_*.cnf")


In [6]:
cnt = 0

for xx in range(len(knob_list)):
    path = "/home/sein/mk_config/ycsb_AA/configs/my_{}.cnf".format(xx)
    # knob_list = glob.glob("/home/sein/2023_EDBT/KCC_tpcc_dataset/my_*.cnf")
    a_all = pd.read_csv(path, sep="=", names=['Sample', 'value'], header=2)
    a_all = a_all.set_index("Sample")
    cur_all_df = a_all.T
    
    if cnt == 0:
        A_config = cur_all_df
    else :
        A_config = pd.concat([A_config, cur_all_df], axis=0)
    cnt += 1
A_config = A_config.reset_index()
A_config = A_config.drop(["index"],axis=1)
A_config = A_config.drop(A_config.columns[[0,1]], axis=1)


A_config

Sample,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,18000,4276224,945000,610000,1,4096000,432013312,0,29,...,159744,520000,91200,8000,63,7680,163577856,57344,104448,1
996,1,57000,5636096,245000,211000,1,8716288,268435456,1,24,...,455936,940000,89600,2000,29,11520,230686720,69632,61440,0
997,1,2000,622592,45000,537000,1,393216,671088640,7,20,...,308480,810000,66000,8000,48,12544,536870912,81920,108544,1
998,0,38000,5505024,700000,598000,0,9125888,834666496,3,17,...,404992,260000,70400,2000,57,3840,218103808,36864,63488,1


In [7]:
samples = pd.concat([A_config,metrics], axis=1)

samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,0.216666,20847397
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,0.322221,12744942
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3789.313102,1076
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,0.166667,12510551
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,0.266666,10686198
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,1,18000,4276224,945000,610000,1,4096000,432013312,0,29,...,91200,8000,63,7680,163577856,57344,104448,1,0.138889,11836756
996,1,57000,5636096,245000,211000,1,8716288,268435456,1,24,...,89600,2000,29,11520,230686720,69632,61440,0,0.149999,16386967
997,1,2000,622592,45000,537000,1,393216,671088640,7,20,...,66000,8000,48,12544,536870912,81920,108544,1,0.227778,19293708
998,0,38000,5505024,700000,598000,0,9125888,834666496,3,17,...,70400,2000,57,3840,218103808,36864,63488,1,0.188888,10529195


In [9]:
### boolean에 해당하는 열 정리

discrete_columns = [samples.columns[0], samples.columns[5],samples.columns[10],samples.columns[13],
                    samples.columns[17],samples.columns[24],samples.columns[30],samples.columns[31],
                    samples.columns[32],samples.columns[34],samples.columns[36],samples.columns[37],
                    samples.columns[58],samples.columns[60],samples.columns[64],samples.columns[68],
                    samples.columns[72],samples.columns[73],samples.columns[74],samples.columns[75],
                    samples.columns[77],samples.columns[80],samples.columns[82],samples.columns[83],
                    samples.columns[90],samples.columns[91],samples.columns[92],samples.columns[93],
                    samples.columns[118],samples.columns[123],samples.columns[124],samples.columns[125],
                    samples.columns[126]]

In [10]:
### continuous 열 정리
all_columns = samples.columns
continuous_columns = all_columns.drop(discrete_columns)

In [11]:
for i in range(len(discrete_columns)):
    a = discrete_columns[i]
    samples = samples.astype({a:'int'})

In [12]:
for i in range(len(continuous_columns)):
    a = continuous_columns[i]
    samples = samples.astype({a:'float'})

### config - metric prediction (with raw data)

In [13]:
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

X_all = np.array(A_config)
Y_all = np.array(metrics)

X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)

y_train_tps = y_train[:,0][:, np.newaxis]
y_train_latency = y_train[:,1][:, np.newaxis]
y_test_tps = y_test[:,0][:, np.newaxis]
y_test_latency = y_test[:,1][:, np.newaxis]
 
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler_latecy = MinMaxScaler().fit(y_train_latency)
Y_scaler_tps  = MinMaxScaler().fit(y_train_tps)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)

scaled_y_train_latecy = Y_scaler_latecy.transform(y_train_latency)
scaled_y_train_tps = Y_scaler_tps.transform(y_train_tps)

scaled_y_test_latecy = Y_scaler_latecy.transform(y_test_latency)
scaled_y_test_tps = Y_scaler_tps.transform(y_test_tps)

# scaled_y_train = np.concatenate([scaled_y_train_latecy, scaled_y_train_tps], 1)
# scaled_y_test = np.concatenate([scaled_y_test_latecy, scaled_y_test_tps], 1)

scaled_y_train = np.concatenate([scaled_y_train_tps, scaled_y_train_latecy], 1)
scaled_y_test = np.concatenate([scaled_y_test_tps, scaled_y_test_latecy], 1)

In [13]:
A_config.shape
# metrics.shape

(1000, 138)

In [18]:
### TabNet
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
from sklearn.metrics import mean_squared_error
from sklearn.preprocessing import MinMaxScaler,StandardScaler
import torch
from sklearn.metrics import r2_score
import warnings
warnings.filterwarnings('ignore')

# X_all = np.array(A_config)
# Y_all = np.array(metrics)

# X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)
# X_scaler = MinMaxScaler().fit(X_train)
# Y_scaler = MinMaxScaler().fit(y_train)

# scaled_X_train = X_scaler.transform(X_train)
# scaled_X_test = X_scaler.transform(X_test)
# scaled_y_train = Y_scaler.transform(y_train)
# scaled_y_test = Y_scaler.transform(y_test)

# Tabnet 모델 생성
regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) ### Basic

# 모델 학습
regressor.fit(X_train=scaled_X_train, y_train=scaled_y_train,
              eval_set=[(scaled_X_test, scaled_y_test)],
              patience=100, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])

predictions = regressor.predict(scaled_X_test)

test_score = mean_squared_error(y_pred = predictions, y_true = scaled_y_test)
# 성능 평가
print('BEST VALID SCORE : ', regressor.best_cost)
# print('MSE_SCORE : ', test_score)
print('R2 SCORE : ' , r2_score(predictions, scaled_y_test, multioutput='variance_weighted'))


epoch 0  | loss: 1.40805 | val_0_mse: 0.15066 |  0:00:00s
epoch 10 | loss: 0.06098 | val_0_mse: 0.05742 |  0:00:01s
epoch 20 | loss: 0.0562  | val_0_mse: 0.05122 |  0:00:01s
epoch 30 | loss: 0.05024 | val_0_mse: 0.05242 |  0:00:02s
epoch 40 | loss: 0.04261 | val_0_mse: 0.05373 |  0:00:03s
epoch 50 | loss: 0.03195 | val_0_mse: 0.05459 |  0:00:04s
epoch 60 | loss: 0.02637 | val_0_mse: 0.0571  |  0:00:05s
epoch 70 | loss: 0.02364 | val_0_mse: 0.06252 |  0:00:06s
epoch 80 | loss: 0.01788 | val_0_mse: 0.05875 |  0:00:07s
epoch 90 | loss: 0.01696 | val_0_mse: 0.04602 |  0:00:08s
epoch 100| loss: 0.01314 | val_0_mse: 0.04374 |  0:00:09s
epoch 110| loss: 0.01281 | val_0_mse: 0.04782 |  0:00:09s
epoch 120| loss: 0.01335 | val_0_mse: 0.04679 |  0:00:10s
epoch 130| loss: 0.00784 | val_0_mse: 0.04622 |  0:00:11s
epoch 140| loss: 0.00718 | val_0_mse: 0.04834 |  0:00:12s
epoch 150| loss: 0.00802 | val_0_mse: 0.04358 |  0:00:13s
epoch 160| loss: 0.00803 | val_0_mse: 0.05088 |  0:00:14s
epoch 170| los

In [49]:
scaled_y_train.shape

(700, 2)

In [16]:
for i in range (len(predictions)):
    print('r2_score : ', r2_score(predictions[i], scaled_y_test[i]))
    # print('Latency_r2_score : ', r2_score(predictions[i][1], scaled_y_test[i][1]))
    
    # print(predictions[i][1],'|', scaled_y_test[i][1])
# print('--------------------')
# print(scaled_y_test)

r2_score :  0.9782563403900324
r2_score :  0.9996099847923541
r2_score :  0.9310220806433419
r2_score :  0.9985115679882028
r2_score :  0.9990994266423234
r2_score :  0.9974280193995283
r2_score :  0.993010343376478
r2_score :  0.24463190779397426
r2_score :  0.7991758159788436
r2_score :  0.9914877565733856
r2_score :  0.9884651492882747
r2_score :  0.9647999431395307
r2_score :  0.9831199116451871
r2_score :  0.9779865737171908
r2_score :  0.9886251981036578
r2_score :  0.9943959939533816
r2_score :  0.9037838406239698
r2_score :  0.9997822711061204
r2_score :  0.9952209531911554
r2_score :  0.9846875067654155
r2_score :  0.18293504909283265
r2_score :  0.999939555138187
r2_score :  0.9252565223822917
r2_score :  0.9987273454696858
r2_score :  0.9971334323995134
r2_score :  0.9793725711908015
r2_score :  0.9923113025811909
r2_score :  0.9990111752567151
r2_score :  0.9975946590469867
r2_score :  0.9841542296222879
r2_score :  0.9995179827336359
r2_score :  0.9718746853663012
r2_score

In [60]:
# np.set_printoptions(threshold=5)

# print(predictions)
# print('-------------------------------')
# print(scaled_y_test)

for i in range(len(predictions)):
    print(predictions[i][1] , '|' , scaled_y_test[i][1])

0.028212171 | 1.6034746226088859e-06
0.4740873 | 0.4770761923036425
0.53928494 | 0.4715801048708036
0.594815 | 0.5021835315673376
0.51900035 | 0.5709834163309614
0.4601705 | 0.4548751953343877
0.51705736 | 0.5171863082508924
0.29538697 | 0.5506477057574182
0.5018295 | 0.5106092562176064
0.029605627 | 2.4809315688698577e-05
0.5319015 | 0.5140742312541791
0.51698893 | 0.7884990693611454
0.5344096 | 0.5436569127305373
-0.0028654523 | 1.282779698087108e-05
0.49963063 | 0.5242157182916011
0.017976433 | 5.919493815131134e-05
0.006161824 | 4.222483172870064e-05
0.49594703 | 0.5519464311198079
0.5306203 | 0.43004013496980387
0.06876944 | 0.0002851066960922076
0.53697515 | 0.4698904880281912
0.048449814 | 9.999445910435962e-05
0.5283218 | 0.5759422952242647
0.5473771 | 0.5070827701128722
0.658903 | 0.5382167687456427
0.5207573 | 0.4888570985197969
0.48328966 | 0.5047949681541032
0.50585884 | 0.48032852878887244
0.5224398 | 0.47484486828435873
0.5461555 | 0.5165302644254367
0.5677705 | 0.5862646

In [113]:
# len(predictions)
predictions[:, 0][0],scaled_y_test[:,0][0]

(0.78120196, 0.9510397321203431)

In [120]:
predictions[:,0][0]

0.78120196

In [19]:
from sklearn.metrics import r2_score

#TPS , Latency

for i in range(2):  
    r2_score_column = r2_score(predictions[:, i], scaled_y_test[:, i])
    print(f'Column {i} R2 Score: {r2_score_column}')
    
    




Column 0 R2 Score: 0.8879543183627583
Column 1 R2 Score: 0.6207964594679785


### LHS SAMPLING

In [20]:
import pandas as pd

knob_info = pd.read_csv('Knob_Information_MySQL_v5.7.csv')

knob_min = knob_info['raw_min']
knob_max = knob_info['raw_max']


In [38]:
import numpy as np

def lhs_sampling(min_val, max_val, num_samples):
    # LHS 샘플 생성
    lhs_samples = np.linspace(min_val, max_val, num_samples, endpoint=False)
    np.random.shuffle(lhs_samples)
    return lhs_samples
samples = lhs_sampling(knob_min, knob_max, 4000)

# 생성된 LHS 샘플 출력
print(samples)
# samples.shape

[[8.75000000e-01 5.25001250e+04 9.17555200e+06 ... 1.14816000e+05
  1.14816000e+05 8.75000000e-01]
 [9.87250000e-01 5.92350128e+04 1.03521188e+07 ... 1.29413888e+05
  1.29413888e+05 9.87250000e-01]
 [5.04000000e-01 3.02404960e+04 5.28685466e+06 ... 6.65681920e+04
  6.65681920e+04 5.04000000e-01]
 ...
 [1.32000000e-01 7.92086800e+03 1.38767565e+06 ... 1.81903360e+04
  1.81903360e+04 1.32000000e-01]
 [1.04500000e-01 6.27089550e+03 1.09942989e+06 ... 1.46140160e+04
  1.46140160e+04 1.04500000e-01]
 [8.01250000e-01 4.80751988e+04 8.40252928e+06 ... 1.05224960e+05
  1.05224960e+05 8.01250000e-01]]


In [257]:
samples.shape

(1000, 138)

In [39]:
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler_latecy = MinMaxScaler().fit(y_train_latency)
Y_scaler_tps  = MinMaxScaler().fit(y_train_tps)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)

scaled_y_train_latecy = Y_scaler_latecy.transform(y_train_latency)
scaled_y_train_tps = Y_scaler_tps.transform(y_train_tps)

scaled_y_test_latecy = Y_scaler_latecy.transform(y_test_latency)
scaled_y_test_tps = Y_scaler_tps.transform(y_test_tps)

In [40]:
### LHS sampling으로 생성한 X값에 대한 scaling

from sklearn.preprocessing import MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

new_X = np.array(samples)
Z_scaler = MinMaxScaler().fit(new_X)
new_X_ = Z_scaler.transform(new_X)
# print(new_X_)

predictions_new = regressor.predict(new_X_) #scaling O


In [41]:
predictions_new

array([[-4.6454044e-04,  5.7587624e-01],
       [ 3.4839360e-03,  5.7733953e-01],
       [ 9.4152335e-04,  5.9531534e-01],
       ...,
       [ 4.1965336e-02,  5.1820815e-01],
       [ 4.4861302e-02,  5.4215348e-01],
       [ 7.8166137e-05,  5.7564116e-01]], dtype=float32)

### 수정한 부분 = inverse_new_ ~ 

In [94]:
### new_metrics_re = 원래 metric + predic_metric
### predictions_new == predict_model.predict(new_sample)

# Y_scaler = MinMaxScaler().fit(predictions_new)
predictions_new_df = pd.DataFrame(predictions_new)

# Y_scaler_tps = MinMaxScaler().fit(predictions_new[:, 0].reshape(-1, 1))
# Y_scaler_latency = MinMaxScaler().fit(predictions_new[:, 1].reshape(-1, 1))

# inverse를 통해서 원래 데이터 형태로
inverse_new_pred_tps = Y_scaler_tps.inverse_transform(predictions_new[:, 0].reshape(-1, 1))
inverse_new_pred_lat = Y_scaler_latecy.inverse_transform(predictions_new[:, 1].reshape(-1, 1))

# inverse_new_pred_tps = Y_scaler_tps.inverse_transform(predictions_new_tps)
# inverse_new_pred_lat = Y_scaler_latecy.inverse_transform(predictions_new_lat)
 

inverse_new_pred_sum = np.concatenate([inverse_new_pred_tps, inverse_new_pred_lat], axis=1)
inverse_new_pred_pd = pd.DataFrame(inverse_new_pred_sum)
inverse_new_pred_pd.rename(columns={0: "tps", 1:"latency"}, inplace=True)
# inverse_new_pred_pd

new_metrics_re = pd.concat([metrics,inverse_new_pred_pd], axis=0)
# new_metrics_re = new_metrics_re.drop(['index'], axis=1)

new_metrics_re = new_metrics_re.reset_index()



In [98]:
# new_metrics_re = new_metrics_re.drop(['index'], axis=1)

new_metrics_re

,tps,latency
0,0.216666,20847397.0
1,0.322221,12744942.0
2,3789.313102,1076.0
3,0.166667,12510551.0
4,0.266666,10686198.0
...,...,...
4995,-24.159760,12362169.0
4996,-2.033373,12927488.0
4997,174.697647,11634883.0
4998,186.753723,12172507.0


In [44]:
new_metrics_re = new_metrics_re.drop(['index'], axis=1)

new_metrics_re


,tps,latency
0,0.216666,2.084740e+07
1,0.322221,1.274494e+07
2,3789.313102,1.076000e+03
3,0.166667,1.251055e+07
4,0.266666,1.068620e+07
...,...,...
4995,-0.006409,5.664213e-01
4996,-0.006072,5.689327e-01
4997,-0.003377,5.631903e-01
4998,-0.003193,5.655788e-01


### 인자 정리
- new_X = samples = LHS Sampling으로 생성한 데이터
<!-- - new_metrics = 생성한 데이터에 대해서 TabNET이 예측한 metrics + 원래 metrics (scaling X) -->
- new_metrics_re = 원래 metric (A_metrics) + 생성한 데이터로 예측한 metric (scaling X)
- new_Samples = 원래 config + 생성한 config (scaling X)
- newnewnew = AutoEncoder에 넣을 수 있는 데이터셋

In [100]:
# inversed_new_config = pd.DataFrame(inverse_transformed_samples)
new_X_pd = pd.DataFrame(new_X)
new_X_pd = round(new_X_pd).astype(int)
for i in range(len(new_X_pd.columns)):
    new_X_pd.rename(columns={new_X_pd.columns[i]: A_config.columns[i]}, inplace=True)   
    
new_Samples = pd.concat([A_config,new_X_pd] , axis=0)

In [101]:
new_Samples = new_Samples.reset_index()
new_Samples = new_Samples.drop(['index'], axis=1)
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,14026,2454185,233750,233750,0,2454185,250987151,2,7,...,122565,233750,23682,2338,16,3830,250987936,31423,31423,0
4996,1,51600,9018327,860000,860000,1,9018327,923417969,6,26,...,450890,860000,86056,8600,55,14090,923418112,112865,112865,1
4997,0,7921,1387676,132000,132000,0,1387676,141733921,1,4,...,69220,132000,13547,1321,9,2163,141734810,18190,18190,0
4998,0,6271,1099430,104500,104500,0,1099430,112206021,1,3,...,54802,104500,10808,1046,8,1712,112206938,14614,14614,0


In [47]:
new_Samples

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,stored_program_cache,sync_binlog,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,489472,200000,13200,4000,18,11520,255852544,28672,94208,0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,374528,50000,91600,4000,17,15360,402653184,110592,71680,1
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,37120,660000,15600,1,9,1024,851443712,8192,122880,0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,427776,430000,11200,10000,9,13568,163577856,16384,83968,0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,388096,880000,11600,4000,25,1536,348127232,126976,57344,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,14026,2454185,233750,233750,0,2454185,250987151,2,7,...,122565,233750,23682,2338,16,3830,250987936,31423,31423,0
4996,1,51600,9018327,860000,860000,1,9018327,923417969,6,26,...,450890,860000,86056,8600,55,14090,923418112,112865,112865,1
4997,0,7921,1387676,132000,132000,0,1387676,141733921,1,4,...,69220,132000,13547,1321,9,2163,141734810,18190,18190,0
4998,0,6271,1099430,104500,104500,0,1099430,112206021,1,3,...,54802,104500,10808,1046,8,1712,112206938,14614,14614,0


In [48]:
new_metrics_re


,tps,latency
0,0.216666,2.084740e+07
1,0.322221,1.274494e+07
2,3789.313102,1.076000e+03
3,0.166667,1.251055e+07
4,0.266666,1.068620e+07
...,...,...
4995,-0.006409,5.664213e-01
4996,-0.006072,5.689327e-01
4997,-0.003377,5.631903e-01
4998,-0.003193,5.655788e-01


### NEW_IDEA
### AE Train set에 추가 (1000 + 10000) AUG == new_Samples

In [102]:
for i in range(len(new_X_pd.columns)):
    new_X_pd.rename(columns={new_X_pd.columns[i]: A_config.columns[i]}, inplace=True)   
    
new_Samples = pd.concat([A_config,new_X_pd] , axis=0)

new_Samples = new_Samples.reset_index()
new_Samples = new_Samples.drop(['index'], axis=1)
# new_Samples

newnewwnew = pd.concat([new_Samples, new_metrics_re], axis = 1)

In [124]:
newnewwnew

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,0.216666,20847397.0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,0.322221,12744942.0
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3789.313102,1076.0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,0.166667,12510551.0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,0.266666,10686198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,14026,2454185,233750,233750,0,2454185,250987151,2,7,...,23682,2338,16,3830,250987936,31423,31423,0,-24.159760,12362169.0
4996,1,51600,9018327,860000,860000,1,9018327,923417969,6,26,...,86056,8600,55,14090,923418112,112865,112865,1,-2.033373,12927488.0
4997,0,7921,1387676,132000,132000,0,1387676,141733921,1,4,...,13547,1321,9,2163,141734810,18190,18190,0,174.697647,11634883.0
4998,0,6271,1099430,104500,104500,0,1099430,112206021,1,3,...,10808,1046,8,1712,112206938,14614,14614,0,186.753723,12172507.0


In [50]:
# new_Samples
new_metrics_re['latency']

0       2.084740e+07
1       1.274494e+07
2       1.076000e+03
3       1.251055e+07
4       1.068620e+07
            ...     
4995    5.664213e-01
4996    5.689327e-01
4997    5.631903e-01
4998    5.655788e-01
4999    5.689189e-01
Name: latency, Length: 5000, dtype: float64

In [51]:
new_Samples.values.shape

(5000, 138)

### 원래 sample + 생성한 sample에 대해 AE가 되는지 확인
- Config와 Metric 따로 스케일링!!

In [103]:

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import  TensorDataset, DataLoader

scaler = MinMaxScaler()
scaled_samples = scaler.fit_transform(new_Samples)
# tps , latency 따로 scaling
scaled_new_metrics_re_tps = scaler.fit_transform(new_metrics_re['tps'].values.reshape(-1, 1))
scaled_new_metrics_re_lat = scaler.fit_transform(new_metrics_re['latency'].values.reshape(-1, 1))

scaled_new_Samples = np.concatenate([scaled_samples,scaled_new_metrics_re_tps,scaled_new_metrics_re_lat], axis = 1)

# tps , Latency 따로 sclaling하고 합친거
X_train, X_test = train_test_split(scaled_new_Samples, test_size=0.2, shuffle=True)

dataset_tr = TensorDataset(torch.tensor(X_train))
dataset_te = TensorDataset(torch.tensor(X_test))

trainloader = torch.utils.data.DataLoader(dataset_tr, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset_te, batch_size=128, shuffle=True)


In [104]:
from sklearn.preprocessing import MinMaxScaler,StandardScaler
from sklearn.model_selection import train_test_split
import torch
from torch.utils.data import  TensorDataset, DataLoader

X_train, X_test = train_test_split(newnewwnew, test_size=0.2, shuffle=True)

scaler = MinMaxScaler()
# new_train = pd.concat([X_train,full_samples])
X_scaler = scaler.fit_transform(X_train)
X_test_scaler = scaler.fit_transform(X_test)


dataset_tr = TensorDataset(torch.tensor(X_scaler))
dataset_te = TensorDataset(torch.tensor(X_test_scaler))

trainloader = torch.utils.data.DataLoader(dataset_tr, batch_size=128, shuffle=True)
testloader = torch.utils.data.DataLoader(dataset_te, batch_size=128, shuffle=True)

In [107]:
### 원본

import torch.nn as nn

class Autoencoder(nn.Module):
  def __init__(self):
    super(Autoencoder, self).__init__()
    self.encoder = nn.Sequential(
        # nn.Linear(140,64),
        # nn.ReLU(),
        # nn.Dropout(p=0.3),
        # nn.Linear(64,32),
        # nn.ReLU(),
        # nn.Linear(32,16), #잠재변수 15개로 줄임 
        # nn.Sigmoid())
        nn.Linear(140,128),
        nn.ReLU(),
        # nn.Dropout(p=0.2),
        nn.Linear(128,64),
        nn.ReLU(),
        # nn.Dropout(p=0.2),
        nn.Linear(64,32),
        # nn.ReLU(),
        # nn.Dropout(p=0.2),
        # nn.Linear(32,16), #잠재변수 15개로 줄임 
        #잠재변수 15개로 줄임 
        nn.Sigmoid())
    
    self.decoder = nn.Sequential(
        # nn.Linear(16,32),
        # nn.ReLU(),
        # nn.Linear(32,64),
        # nn.ReLU(),
        # nn.Linear(64, 140), 
        # nn.Sigmoid()
        # nn.Linear(16,32),
        # nn.ReLU(),
        # nn.Dropout(p=0.2),
        nn.Linear(32,64),
        nn.ReLU(),
        # nn.Dropout(p=0.2),
        nn.Linear(64,128),
        nn.ReLU(),
        nn.Dropout(p=0.2),
        nn.Linear(128, 140), 
        nn.Sigmoid()
    )
  
  #인코더와 디코더 연산을 차례대로 수행하도록 설정 
  def forward(self, x):
    encoded = self.encoder(x)
    decoded = self.decoder(encoded)
    return decoded

In [108]:
# from torch import device
from torch import optim
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
# model =Autoencoder().to(device)
model = Autoencoder().to(device)
# model = Autoencoder()


critertion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.0001)
#lr=1e-4
# device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
trainloss = []
validationloss = []
epoch_list = []

for epoch in range(3000):
  running_loss = 0
  model.train()
  
  for data in trainloader:
    inputs = data[0].float().to(device)
    optimizer.zero_grad()
    outputs = model(inputs) 
    loss = critertion(inputs, outputs)
    loss.backward()
    optimizer.step()
    running_loss += loss.item()
  train_loss = running_loss / len(trainloader)
  trainloss.append(train_loss)
  # epoch_list.append(epoch)
  
  if epoch % 300 == 0:
    total_val_loss = 0
    model.eval()
    with torch.no_grad():
      running_loss = 0
      
      for data in testloader:
        inputs = data[0].float().to(device)
        # optimizer.zero_grad()
        outputs = model(inputs) 
        loss = critertion(inputs, outputs)
        running_loss += loss.item()
        inputs_np = inputs.cpu().detach().numpy()
        outputs_np = outputs.cpu().detach().numpy()
      total_val_loss = running_loss / len(testloader)
      validationloss.append(total_val_loss)
        
    print('[%d] tr_loss : %.3f | val_loss : %.3f' %(epoch +1, train_loss, total_val_loss))
    print('R2_Score :', r2_score(inputs_np, outputs_np, multioutput='variance_weighted') )

[1] tr_loss : 0.126 | val_loss : 0.126
R2_Score : -0.014198556022450648
[301] tr_loss : 0.025 | val_loss : 0.022
R2_Score : 0.7972968248949501
[601] tr_loss : 0.025 | val_loss : 0.022
R2_Score : 0.8451884941513573
[901] tr_loss : 0.025 | val_loss : 0.022
R2_Score : 0.7847171768215299
[1201] tr_loss : 0.024 | val_loss : 0.022
R2_Score : 0.8845696132472239
[1501] tr_loss : 0.023 | val_loss : 0.021
R2_Score : 0.8293247150575676
[1801] tr_loss : 0.023 | val_loss : 0.021
R2_Score : 0.8398522234620229
[2101] tr_loss : 0.022 | val_loss : 0.021
R2_Score : 0.8076171609106997
[2401] tr_loss : 0.022 | val_loss : 0.022
R2_Score : 0.763624122968811
[2701] tr_loss : 0.022 | val_loss : 0.021
R2_Score : 0.8562577840178072


In [185]:
inputs.shape , outputs.shape

(torch.Size([64, 140]), torch.Size([64, 140]))

In [109]:
from sklearn.metrics import r2_score
import numpy as np

# PyTorch Tensor를 NumPy 배열로 변환
inputs_np = inputs.cpu().detach().numpy()
outputs_np = outputs.cpu().detach().numpy()

# r2_score 함수로 계산
r2 = r2_score(inputs_np, outputs_np, multioutput='variance_weighted')

# 결과 출력
print("AE_R2_Score:", r2)


AE_R2_Score: 0.8409458762680269


### Optimization in latent space
- Prediction

In [ ]:
# import torch
# import torch.nn as nn
# import numpy as np

# for data in testloader:
#     inputs = data[0].float().to(device)
#     with torch.no_grad():
#         encoded_vector = model.encoder(inputs)
#         print("Encoded Latent Vector:", encoded_vector)


In [52]:
# print(inputs.shape)
# print(encoded_vector.shape)

torch.Size([88, 140])
torch.Size([88, 32])


In [ ]:
# latent_space = encoded_vector
# print(latent_space.shape)

### TabNet (Optimization과정에서)

In [ ]:
samples.shape

In [117]:
newnewwnew

,automatic_sp_privileges,back_log,binlog_cache_size,binlog_group_commit_sync_delay,binlog_group_commit_sync_no_delay_count,binlog_rows_query_log_events,binlog_stmt_cache_size,bulk_insert_buffer_size,default_week_format,div_precision_increment,...,table_definition_cache,table_open_cache,table_open_cache_instances,thread_cache_size,tmp_table_size,transaction_alloc_block_size,transaction_prealloc_size,updatable_views_with_limit,tps,latency
0,1,24000,7700480,485000,352000,0,655360,356515840,7,1,...,13200,4000,18,11520,255852544,28672,94208,0,0.216666,20847397.0
1,0,15000,622592,140000,701000,1,2637824,595591168,5,8,...,91600,4000,17,15360,402653184,110592,71680,1,0.322221,12744942.0
2,1,8000,7110656,440000,801000,1,8093696,570425344,4,4,...,15600,1,9,1024,851443712,8192,122880,0,3789.313102,1076.0
3,0,34000,8519680,760000,303000,0,10420224,343932928,3,4,...,11200,10000,9,13568,163577856,16384,83968,0,0.166667,12510551.0
4,0,12000,6291456,380000,630000,1,5472256,243269632,1,13,...,11600,4000,25,1536,348127232,126976,57344,0,0.266666,10686198.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4995,0,14026,2454185,233750,233750,0,2454185,250987151,2,7,...,23682,2338,16,3830,250987936,31423,31423,0,-24.159760,12362169.0
4996,1,51600,9018327,860000,860000,1,9018327,923417969,6,26,...,86056,8600,55,14090,923418112,112865,112865,1,-2.033373,12927488.0
4997,0,7921,1387676,132000,132000,0,1387676,141733921,1,4,...,13547,1321,9,2163,141734810,18190,18190,0,174.697647,11634883.0
4998,0,6271,1099430,104500,104500,0,1099430,112206021,1,3,...,10808,1046,8,1712,112206938,14614,14614,0,186.753723,12172507.0


In [158]:
### 원래 데이터 + LHS Sampling data 차원을 32로 줄인 latent space 구하기
### X = latent config (# 1500) , Y = metrics (# 2)

import torch
import pandas as pd
import numpy as np

# scaler_x = MinMaxScaler()
# # samples_scaler = scaler_x.fit_transform(newnewwnew)
# samples_scaler = scaler_x.fit_transform(scaled_new_Samples)

# # samples_np = samples_scaler.values
# samples_torch = torch.Tensor(samples_scaler).to(device)
ex_scaled_new_Samples = torch.Tensor(scaled_new_Samples).to(device)

with torch.no_grad():
    encoded_vector_BO = model.encoder(ex_scaled_new_Samples)
    print("BO'S Encoded Latent Vector:", encoded_vector_BO)

BO'S Encoded Latent Vector: tensor([[0.0575, 0.6138, 0.8563,  ..., 0.4573, 0.8201, 0.7714],
        [0.0909, 0.7782, 0.8587,  ..., 0.5558, 0.6487, 0.7060],
        [0.3300, 0.7510, 0.6827,  ..., 0.7445, 0.4633, 0.6043],
        ...,
        [0.8455, 0.9490, 0.9923,  ..., 0.9910, 0.6729, 0.9879],
        [0.8998, 0.9713, 0.9945,  ..., 0.9948, 0.6149, 0.9923],
        [0.0139, 0.3480, 0.0683,  ..., 0.1807, 0.2270, 0.1046]],
       device='cuda:0')


In [129]:
encoded_vector_BO.shape

torch.Size([5000, 32])

In [ ]:
new_Samples

In [ ]:
# encoded_vector_BO.shape
new_metrics_re

In [82]:
encoded_vector_BO.shape , new_metrics_re.shape

(torch.Size([5000, 32]), (5000, 2))

In [145]:
new_metrics_re

,tps,latency
0,0.216666,20847397.0
1,0.322221,12744942.0
2,3789.313102,1076.0
3,0.166667,12510551.0
4,0.266666,10686198.0
...,...,...
4995,-24.159760,12362169.0
4996,-2.033373,12927488.0
4997,174.697647,11634883.0
4998,186.753723,12172507.0


In [144]:
scaled_lt_y_train , scaled_lt_y_test


# scaled_lt_y_test_tps

(array([[0.00664287, 0.59613637],
        [0.05028929, 0.54034801],
        [0.0060534 , 0.46704272],
        ...,
        [0.00931821, 0.57720615],
        [0.00555245, 0.57567287],
        [0.05296036, 0.55682237]]),
 array([[0.00242117, 0.56991956],
        [0.05578837, 0.57563309],
        [0.00914385, 0.57708539],
        ...,
        [0.00777202, 0.51841332],
        [0.00285107, 0.57200318],
        [0.00607651, 0.57562418]]))

In [159]:
### TabNet
### X = encoded_vector_BO (Scaling O) , Y = metrics (Scaling X)

from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt

X_latent = np.array(encoded_vector_BO.cpu().numpy())
Y_latent = np.array(new_metrics_re)

lt_X_train, lt_X_test, lt_y_train, lt_y_test = train_test_split(X_latent,Y_latent,test_size=0.2, shuffle=True)


y_train_tps = lt_y_train[:,0][:, np.newaxis]
y_train_latecy = lt_y_train[:,1][:, np.newaxis]
y_test_tps = lt_y_test[:,0][:, np.newaxis]
y_test_latecy = lt_y_test[:,1][:, np.newaxis]


Y_scaler_tps  = MinMaxScaler().fit(y_train_tps)
Y_scaler_latecy = MinMaxScaler().fit(y_train_latecy)


scaled_lt_y_train_tps = Y_scaler_tps.transform(y_train_tps)
scaled_lt_y_train_latency = Y_scaler_latecy.transform(y_train_latecy)


scaled_lt_y_test_tps = Y_scaler_tps.transform(y_test_tps)
scaled_lt_y_test_latency = Y_scaler_latecy.transform(y_test_latecy)


scaled_lt_y_train = np.concatenate([scaled_lt_y_train_tps, scaled_lt_y_train_latency], axis = 1)
scaled_lt_y_test = np.concatenate([scaled_lt_y_test_tps, scaled_lt_y_test_latency], axis = 1)

# Tabnet 모델 생성
lt_regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) 
    
# 모델 학습
lt_regressor.fit(X_train=lt_X_train, y_train=scaled_lt_y_train,
              eval_set=[(lt_X_test, scaled_lt_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])

# 테스트 데이터로 예측
lt_predictions = lt_regressor.predict(lt_X_test)

# 성능 평가
print('BEST VALID SCORE : ', lt_regressor.best_cost)
print('R2 SCORE : ' , r2_score(scaled_lt_y_test, lt_predictions, multioutput='variance_weighted'))


# print('MSE_SCORE : ', test_score)



epoch 0  | loss: 0.14202 | val_0_mse: 0.04108 |  0:00:00s
epoch 10 | loss: 0.01168 | val_0_mse: 0.00924 |  0:00:05s
epoch 20 | loss: 0.01118 | val_0_mse: 0.0089  |  0:00:10s
epoch 30 | loss: 0.01102 | val_0_mse: 0.00904 |  0:00:15s
epoch 40 | loss: 0.01094 | val_0_mse: 0.00927 |  0:00:20s
epoch 50 | loss: 0.01056 | val_0_mse: 0.00963 |  0:00:25s
epoch 60 | loss: 0.01056 | val_0_mse: 0.00923 |  0:00:30s
epoch 70 | loss: 0.01101 | val_0_mse: 0.00953 |  0:00:35s

Early stopping occurred at epoch 70 with best_epoch = 20 and best_val_0_mse = 0.0089
BEST VALID SCORE :  0.008897248121006947


TypeError: 'numpy.float64' object is not callable

In [152]:
scaled_lt_y_test.astype

<function ndarray.astype>

In [156]:
# 예측값
lt_predictions = lt_regressor.predict(lt_X_test)

# 관측값
observed_values = scaled_lt_y_test  

# R2 스코어 계산
ssr = np.sum((observed_values - lt_predictions) ** 2)  # SSR
sst = np.sum((observed_values - np.mean(observed_values)) ** 2)  # SST

r2score = 1 - (ssr / sst)
print('R2 Score:', r2score)


R2 Score: 0.863746567585121


In [157]:
r2_score(scaled_lt_y_test, lt_predictions, multioutput='variance_weighted')

TypeError: 'numpy.float64' object is not callable

In [138]:
best_model = lt_regressor
best_r2_score = 1 - best_model.best_cost

# 최상의 모델의 에포크 번호를 얻기
best_epoch = best_model.best_epoch

# 성능 출력
print('BEST VALID SCORE (R2 Score):', best_r2_score)
print('Best Model Epoch:', best_epoch)

BEST VALID SCORE (R2 Score): 0.9886005177530005
Best Model Epoch: 34


In [113]:
scaled_lt_y_test

array([[6.05665900e-03, 5.43692635e-01],
       [1.51600490e-03, 5.64382758e-01],
       [6.29513932e-03, 5.27069012e-01],
       ...,
       [8.41421647e-03, 5.76731784e-01],
       [8.93822908e-01, 6.54752138e-06],
       [6.61032755e-03, 5.95678663e-01]])

In [86]:
lt_y_train

array([[-0.00295807,  0.56790143],
       [-0.00607052,  0.56894195],
       [-0.00598805,  0.57103807],
       ...,
       [-0.00598848,  0.57104397],
       [-0.00604525,  0.56891561],
       [-0.00632609,  0.56781536]])

In [77]:
lt_X_test.shape , scaled_lt_y_test.shape

((1000, 32), (300, 2))

In [75]:
lt_X_train , scaled_lt_y_train

(array([[0.31746525, 0.26020995, 0.06084776, ..., 0.00118747, 0.0038612 ,
         0.7310653 ],
        [0.73175013, 0.9991596 , 0.99310064, ..., 0.917063  , 0.9329298 ,
         0.06691827],
        [0.7671887 , 0.9965803 , 0.9675231 , ..., 0.8392722 , 0.9045803 ,
         0.14178357],
        ...,
        [0.61093044, 0.10683972, 0.47569457, ..., 0.03675497, 0.3021546 ,
         0.10509179],
        [0.31511614, 0.19928375, 0.04729942, ..., 0.00117616, 0.00543735,
         0.75075114],
        [0.6606059 , 0.99821544, 0.9529776 , ..., 0.6966284 , 0.86929256,
         0.6932039 ]], dtype=float32),
 array([[5.17438012e-01, 3.06939138e-05],
        [5.77290124e-01, 6.13878955e-05],
        [3.37706586e-05, 8.56315220e-01],
        ...,
        [6.26258683e-01, 6.67261880e-05],
        [5.23713982e-01, 3.46976998e-05],
        [4.59452928e-01, 3.33631632e-05]]))

In [63]:
print(lt_predictions)
print('-----------------------------------------------')
print(scaled_lt_y_test)

[[ 0.          0.        ]
 [ 0.          0.        ]
 [ 0.          0.        ]
 ...
 [ 0.00071219 -0.00268302]
 [ 0.00072886 -0.00271483]
 [ 0.          0.        ]]
-----------------------------------------------
[[6.59663204e-13 8.52900483e-06]
 [4.10457294e-13 8.52905660e-06]
 [6.36125876e-09 8.52678553e-06]
 ...
 [1.45836742e-04 6.46241822e-06]
 [1.03140924e-04 7.06210675e-06]
 [3.81139128e-13 8.52904590e-06]]


In [64]:
print(lt_X_test)
print('-----------------------------------------------')
print(scaled_lt_y_test)

[[5.23794711e-01 4.41426307e-01 9.18241203e-01 ... 2.50850737e-01
  9.71134543e-01 2.23279044e-01]
 [1.46030575e-01 8.41418207e-01 8.62705827e-01 ... 2.99709439e-01
  2.15932012e-01 9.25122380e-01]
 [6.96771324e-01 3.86776835e-01 6.51102424e-01 ... 4.75303620e-01
  5.61440706e-01 2.82645792e-01]
 ...
 [8.99117906e-03 7.92759180e-01 1.78370182e-03 ... 6.31797314e-02
  1.09356776e-01 4.46293890e-01]
 [3.10261291e-03 8.73375118e-01 1.50288703e-04 ... 2.23087948e-02
  1.10358626e-01 3.98751765e-01]
 [6.00627899e-01 3.80117774e-01 4.84747857e-01 ... 2.17551485e-01
  5.63026488e-01 1.58527270e-01]]
-----------------------------------------------
[[6.59663204e-13 8.52900483e-06]
 [4.10457294e-13 8.52905660e-06]
 [6.36125876e-09 8.52678553e-06]
 ...
 [1.45836742e-04 6.46241822e-06]
 [1.03140924e-04 7.06210675e-06]
 [3.81139128e-13 8.52904590e-06]]


In [49]:
scaled_lt_y_test, lt_predictions

(array([[2.78525694e-13, 8.53510497e-06],
        [4.37804669e-04, 2.36799726e-06],
        [8.77058296e-01, 8.77059645e-01],
        ...,
        [5.86363873e-14, 8.53520716e-06],
        [9.25035584e-01, 9.25036309e-01],
        [3.07845338e-13, 8.53618058e-06]]),
 array([[ 0.00265984,  0.00440948],
        [ 0.02361161,  0.01335548],
        [ 0.6752999 ,  0.6727661 ],
        ...,
        [ 0.00570246,  0.00457435],
        [ 0.680132  ,  0.6671624 ],
        [-0.01929884, -0.01031821]], dtype=float32))

In [44]:
lt_predictions , scaled_lt_y_test

(array([[ 8.6209309e-01,  8.6343294e-01],
        [-1.7992589e-03, -1.6201108e-03],
        [-4.8071379e-05, -4.7473618e-04],
        ...,
        [-6.2339089e-04, -7.5627741e-04],
        [ 4.8741698e-04, -1.4071446e-04],
        [ 8.0739552e-01,  8.0812407e-01]], dtype=float32),
 array([[8.81996862e-01, 8.81998445e-01],
        [2.03473552e-05, 8.23526950e-06],
        [1.00603566e-04, 7.10807332e-06],
        ...,
        [3.22665924e-13, 8.53940563e-06],
        [1.78271072e-04, 6.01720332e-06],
        [7.97495617e-01, 7.97496896e-01]]))

In [ ]:
lt_X_test.shape, lt_y_test.shape

In [ ]:
lt_X_train

In [ ]:
scaled_lt_y_train

In [ ]:
# predictions, scaled_y_test
from sklearn.metrics import r2_score

# r2_score(lt_predictions, lt_y_test)
r2_score(scaled_lt_y_test, lt_predictions)



In [35]:
latent_pd = pd.DataFrame(encoded_vector_BO.cpu().numpy())
latent_pd_T = latent_pd.T
latent_pd_T.index
# latent_pd_T

RangeIndex(start=0, stop=32, step=1)

In [ ]:
latent_pd_T

In [37]:
### BO 코드
from sklearn.preprocessing import MinMaxScaler
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

class BO(object):
    def __init__(self, iteration, configs, metrics, config_info_path=None):
        self.iteration = iteration
        self.configs = configs
        self.metrics = metrics
        self.config_info_path = config_info_path
        # self.min_max_same_knobs = []
        
        self._get_config_info()
        self._init_pbounds()
    
    def _get_config_info(self):
        if self.config_info_path is None:
            self.config_info = pd.read_csv('/home/sein/mk_config/Knob_Information_MySQL_v5.7.csv', index_col=0)
        else:
            self.config_info = pd.read_csv(self.config_info_path, index_col=0)
        
        # if self.top_z_knob is not None:
        #     self.config_info = self.config_info.loc[self.top_z_knob]
        #     self.configs = self.configs[self.top_z_knob]
            

#     def _get_history(self):
#         self.history_configs = self.smac.runhistory.get_configs()
    
    def _init_pbounds(self):
        self.pbounds = {}
        
        for v in latent_pd_T.index:
            # self.pbounds[v] = (0,18)
            self.pbounds[str(v)] = (0, 1)
            
    
    def train_regression_model(self):
        X_all = np.array(self.configs)
        Y_all = np.array(self.metrics)
        
        cnt = 0
        while(True):
            X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)         
            
            self.X_scaler = MinMaxScaler().fit(X_train)
            self.y_scaler = MinMaxScaler().fit(y_train)

            # scaled_X_train = self.X_scaler.transform(X_train)
            # scaled_X_test = self.X_scaler.transform(X_test)
            scaled_y_train = self.y_scaler.transform(y_train)
            scaled_y_test = self.y_scaler.transform(y_test)
            
            
            estimator = TabNetRegressor(verbose = 100, seed = 42, optimizer_fn=torch.optim.AdamW)

            estimator.fit(X_train=X_train, y_train=scaled_y_train,
              eval_set=[(X_test, scaled_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])
            pred = estimator.predict(X_test)
            accuracy = mean_squared_error(y_pred = pred, y_true = scaled_y_test)
            r2_score_ = r2_score(scaled_y_test, pred)

            cnt += 1
            print(cnt, accuracy)
            print(r2_score_)
            if r2_score_ > 0.90 or cnt > 10:
                break


            # print('BEST VALID SCORE : ', regressor.best_cost)

        self.model = estimator
    
    def _target_function(self, **kwargs):
        x = np.fromiter(kwargs.values(), dtype=float)        
        # scaled_X = self.X_scaler.transform([x])
        res = self.model.predict(x)
        res = res[:,0] / res[:,1]
        # for name in self.config_info.index:
        #     knob = self.config_info.loc[name]
        #     knob_min = knob['raw_min']
        #     knob_max = knob['raw_max']
            
            # if self.reduce_search_space:
            #     knob_min = self.best_configs[name].min()
            #     knob_max = self.best_configs[name].max()
            #     if knob_min == knob_max:
            #         self.min_max_same_knobs.append(name)
            #         knob_max += 1
            # with torch.no_grad():
            #     scaled_X_t = torch.tensor(scaled_X)
            #     scaled_X_t = scaled_X_t.to('cuda:0')
            #     if model.decoder(scaled_X_t) < knob_min:
            #         res == 0
            #     elif model.decoder(scaled_X_t) > knob_max:
            #         res == 0
        
        
        return res.squeeze()
        # return res
    
    
    def tune(self):
        self.optimizer = BayesianOptimization(f=self._target_function, pbounds=self.pbounds, verbose=2, random_state=2)
        # self.optimizer = BayesianOptimization(f=self._target_function, verbose=2, random_state=2)
        self.acquisition_function = UtilityFunction(kind="ei", xi=0.01)
        self.optimizer.maximize(n_iter=self.iteration, acquisition_function=self.acquisition_function)
    
    
    def plot_history(self):
        self.y_obs = - np.array([res["target"] for res in self.optimizer.res])
        
        self.his_inc = []
        inc = np.inf
        ## Get minimum results on each iteration
        for res in self.y_obs:
            if res < inc:
                inc = res
            self.his_inc.append(inc)
#             res.append(his_res)

        plt.plot(self.his_inc)
        plt.xlabel('Iteration')
        plt.ylabel('result')
        plt.show()

### 밑에 다 돌린거고 decoding해서 값 확인해보기

In [ ]:
tuner1 = BO(iteration=1000, 
           configs=encoded_vector_BO.cpu().numpy(),
           metrics=new_metrics_re
           )
tuner1.train_regression_model()
tuner1.tune()
# tuner1.plot_history()

In [47]:
ex_data = [0.2515,  0.3851, 0.1983, 0.2072, 0.04203, 0.6034, 0.7527, 0.309, 0.01957, 0.107,
           0.6571, 0.7406, 0.9737, 0.1054, 0.5471, 0.467, 0.3135, 0.9268, 0.1925, 0.1914, 0.7365,
           0.4656, 0.9693, 0.5216, 0.2464, 0.4485, 0.282, 0.4303, 0.4483, 0.5755, 0.1848, 0.99]

In [56]:
ex_data = [0.2168, 0.0, 0.138, 1.0, 0.3111, 0.333, 0.6079, 0.1281, 0.0, 0.2796, 0.3718, 0.3238, 0.5304,
           0.474, 0.1646, 0.5807, 0.9193, 0.09729, 0.1823, 0.1282, 0.3789, 0.4876, 0.9011, 0.1251, 0.3144,
           0.04871, 0.8719, 1.0, 0.5318, 0.5375, 0.7821, 0.865]

In [52]:
ex_data = [0.8619, 0.5249, 0.6002, 0.9991, 0.3543, 0.5662, 0.6583, 0.9311, 0.7842, 0.05524,
           0.02226, 0.5493, 0.7891, 0.3761, 0.1813, 0.631, 0.4767, 0.1582, 0.9703, 0.9681,
           0.9885, 0.0, 0.8136, 0.3381, 0.3356, 0.998, 0.8706, 0.2109, 0.389, 0.1532, 0.0, 0.9467  ]

In [58]:
ex_data = torch.tensor(ex_data)
ex_data = ex_data.to('cuda:0')  # ex_data를 GPU로 이동

with torch.no_grad():
    decode_value = model.decoder(ex_data)
    print("Decoded Value:", decode_value)

Decoded Value: tensor([6.4824e-01, 8.2373e-01, 5.4986e-01, 4.6874e-01, 5.0770e-01, 1.3226e-07,
        7.3256e-01, 4.1707e-01, 2.4584e-01, 5.5880e-01, 4.5944e-03, 3.6874e-01,
        4.4241e-01, 1.0000e+00, 4.1733e-01, 7.0821e-01, 4.6285e-01, 1.4779e-01,
        1.9396e-01, 3.4814e-01, 4.1392e-01, 3.0256e-01, 3.6378e-01, 7.0773e-01,
        8.7684e-01, 4.3775e-01, 4.7638e-01, 4.9350e-01, 5.1865e-01, 1.3259e-01,
        6.5164e-01, 4.1798e-01, 3.8149e-01, 6.0668e-01, 2.6327e-01, 6.7646e-01,
        2.9503e-05, 9.9994e-01, 5.3028e-01, 6.0091e-01, 6.0628e-01, 7.1926e-01,
        4.7253e-01, 5.9855e-01, 2.6013e-01, 4.4548e-01, 3.2324e-01, 5.8244e-01,
        8.3967e-01, 1.3947e-01, 5.7830e-01, 6.3821e-01, 4.8185e-01, 4.6733e-01,
        3.5292e-01, 6.0820e-01, 5.9133e-01, 4.2697e-01, 5.0241e-01, 1.8115e-01,
        3.1135e-01, 4.6330e-01, 3.3856e-01, 3.1810e-01, 9.9703e-01, 3.5718e-01,
        5.0331e-01, 2.6912e-01, 9.9668e-01, 2.4544e-01, 5.7020e-01, 4.3492e-01,
        9.9931e-01, 2.790

In [59]:
for x in decode_value:
    print(x)

tensor(0.6482, device='cuda:0')
tensor(0.8237, device='cuda:0')
tensor(0.5499, device='cuda:0')
tensor(0.4687, device='cuda:0')
tensor(0.5077, device='cuda:0')
tensor(1.3226e-07, device='cuda:0')
tensor(0.7326, device='cuda:0')
tensor(0.4171, device='cuda:0')
tensor(0.2458, device='cuda:0')
tensor(0.5588, device='cuda:0')
tensor(0.0046, device='cuda:0')
tensor(0.3687, device='cuda:0')
tensor(0.4424, device='cuda:0')
tensor(1., device='cuda:0')
tensor(0.4173, device='cuda:0')
tensor(0.7082, device='cuda:0')
tensor(0.4629, device='cuda:0')
tensor(0.1478, device='cuda:0')
tensor(0.1940, device='cuda:0')
tensor(0.3481, device='cuda:0')
tensor(0.4139, device='cuda:0')
tensor(0.3026, device='cuda:0')
tensor(0.3638, device='cuda:0')
tensor(0.7077, device='cuda:0')
tensor(0.8768, device='cuda:0')
tensor(0.4378, device='cuda:0')
tensor(0.4764, device='cuda:0')
tensor(0.4935, device='cuda:0')
tensor(0.5186, device='cuda:0')
tensor(0.1326, device='cuda:0')
tensor(0.6516, device='cuda:0')
tensor(0

In [60]:
import math


real_values = [math.exp(x) for x in decode_value]  #지수함수 없는 상태

print(real_values)

[1.912178995759835, 2.2789735949138907, 1.7330114134427155, 1.597984258355564, 1.6614631920616965, 1.000000132257975, 2.0803938716322485, 1.5175015913200782, 1.2786893807613118, 1.7485660432166548, 1.0046049419613148, 1.4459078578027134, 1.5564478061710612, 2.718281828459045, 1.5179080834726542, 2.030358978378201, 1.5885974568696961, 1.1592693894888686, 1.2140511525883404, 1.4164299047781657, 1.5127391014767344, 1.3533162940503463, 1.4387626148435155, 2.029378115761251, 2.403296714663861, 1.5492245906112134, 1.6102346297956847, 1.638033859426608, 1.6797570348813826, 1.1417782282184568, 1.9186761979531235, 1.5188868792509453, 1.4644623873146607, 1.8343245848565575, 1.301183794193486, 1.966903668002052, 1.000029503009967, 2.7181259675492755, 1.6994156630544832, 1.8237857419154884, 1.8335980932324465, 2.0529127117365844, 1.6040440646668888, 1.8194739103291697, 1.2971037858974417, 1.5612458147856194, 1.3815925051603606, 1.7903941420867773, 2.31559663458674, 1.1496588116366608, 1.7830083567

In [61]:
### 원본

import numpy as np

# real_values 리스트를 NumPy 배열로 변환
real_values = np.array(real_values)
rescaled_actual = scaler.inverse_transform(real_values.reshape(1,-1))

In [62]:
# rescaled_actual.shape
# real_values.shape
rescaled_actual


array([[1.91217900e+00, 1.36737137e+05, 1.81239031e+07, 1.58621046e+06,
        1.65349441e+06, 1.00000013e+00, 2.17419152e+07, 1.62723437e+09,
        8.95082567e+00, 5.24569813e+01, 1.00460494e+00, 1.44590786e+04,
        1.08951346e+01, 2.71828183e+00, 2.73223455e+02, 3.14553847e+01,
        1.58859746e+03, 1.15926939e+00, 4.94455261e+03, 7.24795681e+02,
        1.50668815e+06, 1.34684308e+03, 2.42466357e+10, 1.01468906e+02,
        2.40329671e+00, 1.54922459e+03, 1.61023463e+02, 1.47423047e+01,
        1.25981778e+02, 1.13593369e+05, 1.91867620e+00, 1.51888688e+00,
        1.46446239e+00, 1.75089213e+02, 1.30118379e+00, 1.55199001e+08,
        1.00002950e+00, 2.71812597e+00, 4.19559175e+02, 2.91805719e+01,
        1.75023828e+04, 2.19107302e+09, 5.07253660e+01, 2.33910458e+09,
        1.29413275e+04, 1.55543959e+05, 1.47938954e+09, 1.91159646e+09,
        3.72651498e+04, 1.14683026e+04, 1.77517827e+02, 1.89309192e+02,
        1.61906517e+05, 1.59573339e+07, 2.44461942e+10, 1.102277

In [65]:
pd.options.display.float_format = '{:.9f}'.format #지수함수 없이 출력하는 option

rescaled_actual_pd = pd.DataFrame(rescaled_actual)
rescaled_actual_pd

,0,1,2,3,4,5,6,7,8,9,...,130,131,132,133,134,135,136,137,138,139
0,1.912178996,136737.136721239,18123903.122629706,1586210.462996942,1653494.412909326,1.000000132,21741915.203722715,1627234367.592443228,8.950825665,52.456981296,...,170349.430927770,13466.212357232,75.087152332,35295.804527475,1802430661.499608755,211210.659261347,148492.394305443,1.075382621,119577903103.999984741,1082121774956543.875000000


In [66]:
# len(A_config.columns)
rescaled_actual_pd.iloc[0][1]

136737.13672123852

In [67]:
for i in range (len(A_config.columns)):
    print('{} = {}'.format(A_config.columns[i], round(rescaled_actual_pd.iloc[0][i])))

automatic_sp_privileges  = 2
back_log  = 136737
binlog_cache_size  = 18123903
binlog_group_commit_sync_delay  = 1586210
binlog_group_commit_sync_no_delay_count  = 1653494
binlog_rows_query_log_events  = 1
binlog_stmt_cache_size  = 21741915
bulk_insert_buffer_size  = 1627234368
default_week_format  = 9
div_precision_increment  = 52
end_markers_in_json  = 1
eq_range_index_dive_limit  = 14459
expire_logs_days  = 11
explicit_defaults_for_timestamp  = 3
flush_time  = 273
ft_min_word_len  = 31
ft_query_expansion_limit  = 1589
general_log  = 1
group_concat_max_len  = 4945
innodb_adaptive_hash_index_parts  = 725
innodb_adaptive_max_sleep_delay  = 1506688
innodb_autoextend_increment  = 1347
innodb_buffer_pool_size  = 24246635739
innodb_change_buffer_max_size  = 101
innodb_cmp_per_index_enabled  = 2
innodb_commit_concurrency  = 1549
innodb_compression_failure_threshold_pct  = 161
innodb_compression_level  = 15
innodb_compression_pad_pct_max  = 126
innodb_concurrency_tickets  = 113593
innodb_dead

### Data Prepare (Metric)

In [ ]:
inputs

In [ ]:
latent_pd = pd.DataFrame(latent_space.cpu().numpy())

In [ ]:
latent_pd

In [ ]:
latent_pd.max()

### Tabnet type 나눈거

In [ ]:
### TabNet
from pytorch_tabnet.tab_model import TabNetRegressor
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
import matplotlib.pyplot as plt

X_all = np.array(latent_space.cpu().numpy())
Y_all = np.array(new_metric)

### 데이터 나누기
### boolean에 해당하는 열 정리

discrete_columns = [samples.columns[0], samples.columns[5],samples.columns[10],samples.columns[13],
                    samples.columns[17],samples.columns[24],samples.columns[30],samples.columns[31],
                    samples.columns[32],samples.columns[34],samples.columns[36],samples.columns[37],
                    samples.columns[58],samples.columns[60],samples.columns[64],samples.columns[68],
                    samples.columns[72],samples.columns[73],samples.columns[74],samples.columns[75],
                    samples.columns[77],samples.columns[80],samples.columns[82],samples.columns[83],
                    samples.columns[90],samples.columns[91],samples.columns[92],samples.columns[93],
                    samples.columns[118],samples.columns[123],samples.columns[124],samples.columns[125],
                    samples.columns[126]]

discrete_idxs = [0, 5, 10, 13, 17, 24, 30, 31, 32, 34, 36, 37, 58, 60, 64, 68, 72, 73, 74, 75, 77, 80, 82,
                 83, 90, 91, 92, 93, 118, 123, 124, 125, 126]

discrete_dims = 

X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.2, shuffle=True)
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler = MinMaxScaler().fit(y_train)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)
scaled_y_train = Y_scaler.transform(y_train)
scaled_y_test = Y_scaler.transform(y_test)

# Tabnet 모델 생성
regressor = TabNetRegressor(verbose = 10,seed = 42,optimizer_fn=torch.optim.AdamW) ### Basic

# 모델 학습
regressor.fit(X_train=scaled_X_train, y_train=scaled_y_train,
              eval_set=[(scaled_X_test, scaled_y_test)],
              patience=50, 
              batch_size = 128,
              max_epochs=10000,
              eval_metric=['mse'])

# regressor.fit(X_train=scaled_X_train, y_train=scaled_y_train)



# predictions_array =[]
# CV_score_array    =[]

# CV_score_array.append(regressor.best_cost)
# predictions_array.append(np.expm1(regressor.predict(scaled_X_test)))

# predictions = np.mean(predictions_array,axis=0)

# 테스트 데이터로 예측
# y_pred = rf_regressor.predict(scaled_X_test)
predictions = regressor.predict(scaled_X_test)

test_score = mean_squared_error(y_pred = predictions, y_true = scaled_y_test)
# 성능 평가
print('BEST VALID SCORE : ', regressor.best_cost)
print('MSE_SCORE : ', test_score)


In [ ]:
from sklearn.model_selection import GridSearchCV, RandomizedSearchCV
from xgboost import XGBRegressor
from sklearn.model_selection import KFold

X_all = np.array(new_knobs)
Y_all = np.array(new_metric)

est = XGBRegressor()

param_grid = {
    'n_estimators' : [15000, 20000, 25000],
    'max_depth' : [12,13,14,15],
    'learning_rate' : [0.1, 0.001],  
}

grid_search = GridSearchCV(estimator=est, param_grid=param_grid, scoring='accuracy', cv = 3)

grid_search.fit(X_all,Y_all)

print("Best Hyperparameters: ", grid_search.best_params_)
print("Best Score: ", grid_search.best_score_)

### ㅎㅎ..

In [ ]:
#####Shap_value

###XGBRegressor

from sklearn.preprocessing import MinMaxScaler,StandardScaler
from xgboost import XGBRegressor
from sklearn.model_selection import train_test_split
from sklearn.multioutput import MultiOutputRegressor
# from sklearn.metrics import explained_variance_score
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.metrics import mean_squared_error, r2_score, precision_score
import numpy as np

X_all = np.array(new_knobs)
Y_all = np.array(new_metric)

X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.4, shuffle=True)
X_scaler = MinMaxScaler().fit(X_train)
Y_scaler = MinMaxScaler().fit(y_train)

scaled_X_train = X_scaler.transform(X_train)
scaled_X_test = X_scaler.transform(X_test)
scaled_y_train = Y_scaler.transform(y_train)
scaled_y_test = Y_scaler.transform(y_test)

# xgb_model = xgb.XGBRegressor(n_estimators=100, learning_rate=0.08, gamma=0, subsample=0.75,
#                            colsample_bytree=1, max_depth=7)

estimator = XGBRegressor(
    objective = 'reg:squarederror',
    random_state = 2, n_estimators = 15000, max_depth = 12, learning_rate = 0.01)

my_model_pred = MultiOutputRegressor(estimator = estimator).fit(scaled_X_train, scaled_y_train)


pred = my_model_pred.predict(scaled_X_test)

#rmse
print('RMSE : ',mean_squared_error(scaled_y_test, pred, squared=False))
print('R2_SCORE : ', r2_score(scaled_y_test, pred))
# print('PCC : ', precision_score(y_test, pred, average='weighted'))

# rmse r2 pcc




- Optimization

In [ ]:
from bayes_opt import BayesianOptimization
from bayes_opt import UtilityFunction

class BO(object):
    def __init__(self, iteration, configs, metrics,config_info_path=None):
        self.iteration = iteration
        self.configs = configs
        self.metrics = metrics
        self.config_info_path = config_info_path
        # self.reduce_search_space = reduce_search_space
        # self.top_z_knob = top_z_knob
        # self.best_configs = best_configs
        self.min_max_same_knobs = []
        
        self._get_config_info()
        self._init_pbounds()
    
    def _get_config_info(self):
        if self.config_info_path is None:
            self.config_info = pd.read_csv('/home/sein/mk_config/Knob_Information_MySQL_v5.7.csv', index_col=0)
        else:
            self.config_info = pd.read_csv(self.config_info_path, index_col=0)
        
        # if self.top_z_knob is not None:
        #     self.config_info = self.config_info.loc[self.top_z_knob]
        #     self.configs = self.configs[self.top_z_knob]
            

#     def _get_history(self):
#         self.history_configs = self.smac.runhistory.get_configs()
    
    def _init_pbounds(self):
        self.pbounds = {}
        
        for name in self.config_info.index:
            knob = self.config_info.loc[name]
            knob_type = knob['type']
            knob_min = knob['raw_min']
            knob_max = knob['raw_max']
            
            # if self.reduce_search_space:
            #     knob_min = self.best_configs[name].min()
            #     knob_max = self.best_configs[name].max()
            #     if knob_min == knob_max:
            #         self.min_max_same_knobs.append(name)
            #         knob_max += 1
            
            if knob_type == 'boolean' or knob_type == 'integer':
                self.pbounds[name] = (int(knob_min), int(knob_max))
            elif knob_type == 'float':
                self.pbounds[name] = (float(knob_min), float(knob_max))
    
    def train_regression_model(self):
        X_all = np.array(self.configs)
        Y_all = np.array(self.metrics)

        cnt = 0
        while(True):
            X_train, X_test, y_train, y_test = train_test_split(X_all,Y_all,test_size=0.3, shuffle=True)
            self.X_scaler = MinMaxScaler().fit(X_train)
            self.y_scaler = MinMaxScaler().fit(y_train)

            scaled_X_train = self.X_scaler.transform(X_train)
            scaled_X_test = self.X_scaler.transform(X_test)
            scaled_y_train = self.y_scaler.transform(y_train)
            scaled_y_test = self.y_scaler.transform(y_test)

            estimator = XGBRegressor(
                objective = 'reg:squarederror',
                random_state = 2, n_estimators = 15000, max_depth = 12, learning_rate = 0.01)

            my_model_pred = MultiOutputRegressor(estimator = estimator).fit(scaled_X_train, scaled_y_train)
            pred = my_model_pred.predict(scaled_X_test)
            accuracy = r2_score(scaled_y_test, pred)

            cnt += 1
            print(cnt, accuracy)
            if accuracy > 0.45 or cnt > 10:
                break

        #rmse
        print('RMSE : ',mean_squared_error(scaled_y_test, pred, squared=False))
        print('R2_SCORE : ', r2_score(scaled_y_test, pred))

        self.model = my_model_pred
    
    def _target_function(self, **kwargs):
        x = np.fromiter(kwargs.values(), dtype=float)        
        scaled_X = self.X_scaler.transform([x])
        
        res = self.model.predict(scaled_X)
        res = res[:,0] / res[:,1]
        
        return res.squeeze()
#         print(y)
        
#         X = pd.DataFrame(config.get_dictionary().values(), index=self.config_info.index).T
#         X = np.array(X)
#         scaled_X = self.X_scaler.transform(X)
        
#         res = self.model.predict(scaled_X)
#         res = res[:,1] / res[:,0]
        
#         return res # a smaller result means better
    
    
    def tune(self):
        self.optimizer = BayesianOptimization(f=self._target_function, pbounds=self.pbounds, verbose=2, random_state=2)
        # self.optimizer = BayesianOptimization(f=self._target_function, verbose=2, random_state=2)
        self.acquisition_function = UtilityFunction(kind="ei", xi=0.01)
        self.optimizer.maximize(n_iter=self.iteration, acquisition_function=self.acquisition_function)
    
    
    def plot_history(self):
        self.y_obs = - np.array([res["target"] for res in self.optimizer.res])
        
        self.his_inc = []
        inc = np.inf
        ## Get minimum results on each iteration
        for res in self.y_obs:
            if res < inc:
                inc = res
            self.his_inc.append(inc)
#             res.append(his_res)

        plt.plot(self.his_inc)
        plt.xlabel('Iteration')
        plt.ylabel('result')
        plt.show()

In [ ]:
tuner1 = BO(iteration=150, 
           configs=new_knobs,
           metrics=new_metric
           )
tuner1.train_regression_model()
tuner1.tune()
# tuner1.plot_history()